In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import glob
import cv2
from tensorflow.keras import models,layers,Sequential
from sklearn.model_selection import train_test_split
import random
from skimage.filters import gabor_kernel
from scipy import ndimage as ndi
from sklearn.utils import shuffle

In [2]:
class_names = ["EMCI","LMCI","MCI","AD","CN"]
class_index=[0,1,2,3,4]

In [3]:
path=r"G:\dataset\Alzheimers-ADNI"

In [4]:
'''reading the images'''
data=dict()
# trainData=[]
# trainDataLables=[]
coo_matrix=[]
for name in class_names:
    images = [cv2.imread(file,cv2.COLOR_BGR2GRAY) for file in glob.glob(path+"\\train\\"+name+r'\*.jpg')]
    coo_matrix.append({name:len(images)})
    temp=[]
    for i in images:
        i=cv2.resize(i,(256, 256))
        temp.append(np.asarray(i).reshape(256, 256,1))
    data[name]=temp

In [5]:
'''balancing imbalanced data'''
print('before')
print(coo_matrix)
max_count=0
for name in class_names:
    if len(data[name])>max_count:
        max_count=len(data[name])
print(max_count)

for name in class_names:
    n=len(data[name])
    for t in range(n,max_count):
        data[name].append(data[name][random.randint(0, n)])

coo_matrix=[]
for name in class_names:
    n=len(data[name])
    coo_matrix.append({name:n})

print('after')
print(coo_matrix)

before
[{'EMCI': 204}, {'LMCI': 61}, {'MCI': 196}, {'AD': 145}, {'CN': 486}]
486
after
[{'EMCI': 486}, {'LMCI': 486}, {'MCI': 486}, {'AD': 486}, {'CN': 486}]


In [6]:
output_size=len(class_names)

In [7]:
trainData=[]
trainDataLables=[]

for name in class_names:
    for i in data[name]:
        trainData.append(i.flatten())
        trainDataLables.append(name)

trainData=np.array(trainData)
trainDataLables=np.asarray(trainDataLables)
trainData,trainDataLables=shuffle(trainData,trainDataLables)

In [8]:
lableId=[]
for i in range(len(trainDataLables)):
    for j in range(len(class_names)):
        if trainDataLables[i]==class_names[j]:
            lableId.append(j)

trainLables=tf.keras.utils.to_categorical(lableId, num_classes=len(class_names), dtype='float32')

In [9]:
X_train, X_test, y_train, y_test = train_test_split(trainData,
                                                    trainLables,
                                                    test_size=.25,
                                                    random_state=1234123)

In [10]:
X_val, X_test, y_val, y_test = train_test_split(X_test,
                                                    y_test,
                                                    test_size=.5,
                                                    random_state=1234123)

In [11]:
def dense_block(units, dropout_rate):
    block = Sequential([
        layers.Dense(units, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(dropout_rate)
    ])
    
    return block

In [12]:
model = models.Sequential([
    layers.Dense(4096, activation='relu',input_shape =(65536,)),
    layers.Dense(2048, activation='relu'),
    layers.Dense(1024, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(.3),
    layers.Dense(512, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(.2),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(.2),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(.2),
    layers.Dense(output_size, activation='softmax')
])
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4096)              268439552 
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              8390656   
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
batch_normalization (BatchNo (None, 1024)              4096      
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               524800    
_________________________________________

In [13]:
METRICS = [tf.keras.metrics.AUC(name='auc')]
model.compile(
        optimizer='adam',
        loss=tf.keras.losses.categorical_crossentropy,
        metrics=METRICS
    )

In [14]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 **(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(0.01, 20)

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("alzheimer_model.h5",
                                                    save_best_only=True)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10,
                                                     restore_best_weights=True)

In [15]:
EPOCHS=100
history = model.fit(
    X_train,y_train,
    validation_data=(X_val,y_val),
    epochs=EPOCHS,verbose=2
)

Train on 1822 samples, validate on 304 samples
Epoch 1/100


In [ ]:
model.evaluate(X_test, y_test)

Error: Session cannot generate requests